# get pdf

In [1]:
from get_pdf import get_jaist_lecture_pdf

In [2]:
YEAR = 2022

In [3]:
file_names = get_jaist_lecture_pdf(year=YEAR)

In [4]:
file_names

['pdfs/calendar2022_4-8-1.pdf',
 'pdfs/calendar2022_9-12.pdf',
 'pdfs/calendar2022_1-3-1.pdf']

In [5]:
ls pdfs

calendar2021_1-3.pdf     calendar2021_9-12_4.pdf  calendar2022_9-12.pdf
calendar2021_4-8-1.pdf   calendar2022_1-3-1.pdf
calendar2021_9-12.pdf    calendar2022_4-8-1.pdf


# convert pdf to xlsx

このサイト使って変換する

[sodapdf](https://www.sodapdf.com/pdf-converter/)

In [8]:
ls xlsxs/

SodaPDF-processed-calendar2022_1-3-1.xlsx
SodaPDF-processed-calendar2022_4-8-1.xlsx
SodaPDF-processed-calendar2022_9-12.xlsx
calendar2021_1-3.xlsx
calendar2021_1-3_old.xlsx
calendar2021_4-8-1.xlsx
calendar2021_9-12.xlsx
calendar2021_9-12_4.xlsx


In [9]:
xlsx_fnames = [
 'xlsxs/SodaPDF-processed-calendar2022_4-8-1.xlsx',
 'xlsxs/SodaPDF-processed-calendar2022_9-12.xlsx',
 'xlsxs/SodaPDF-processed-calendar2022_1-3-1.xlsx',
]

# convert pdf to pandas_dataframe

In [10]:
import read_xlsx

In [ ]:
#import importlib
#importlib.reload(read_xlsx)

<module 'read_xlsx' from '/Users/saitoyutaro/jaist-tokyo-lecture/lib/read_xlsx.py'>

In [11]:
df = read_xlsx.preprocess_xlsx(xlsx_fnames)

In [12]:
df.reset_index().groupby(['year','month'])['day'].nunique().reset_index()

,year,month,day
0,2022,4,30
1,2022,5,31
2,2022,6,30
3,2022,7,31
4,2022,8,31
5,2022,9,30
6,2022,10,31
7,2022,11,30
8,2022,12,31
9,2023,1,31


## get unique lecture

In [13]:
leclist = read_xlsx.get_unique_lecture(df)

In [20]:
#leclist

## get lecture day and time

In [15]:
import pandas as pd
from datetime import datetime

In [16]:
lecdict = {}
for lec in leclist:
    lectime = []
    for index, row in df[df==lec].dropna(how='all').iterrows():
        for col in row.dropna().index:
            lectime.append([index,col])
    lecdict.update({lec: lectime})

In [19]:
# lecdict

In [17]:
df_lec = pd.DataFrame.from_dict(lecdict, orient='index').stack().reset_index(0)
df_lec.columns = ['title', 0]

srs_date =  df_lec[0].str[0].str
df_lec['Start Date'] = df_lec['End Date'] = df_lec[0].str[0].apply(lambda datetuple: datetime(*datetuple)).dt.strftime('%Y/%m/%d')

In [22]:
d={int(i[0]): i.split('\n')[1].split(' - ') for i in [
    '1限\n09:20 - 11:00', '2限\n11:10 - 12:50', '3限\n13:50 - 15:30',
    '4限\n15:40 - 17:20', '5限\n17:30 - 19:10', '6限\n18:30 - 20:10',
    '7限\n20:15 - 21:55', '8変則時間割（石川の講義を配信）\n17:10 - 18:50']}

d

{1: ['09:20', '11:00'],
 2: ['11:10', '12:50'],
 3: ['13:50', '15:30'],
 4: ['15:40', '17:20'],
 5: ['17:30', '19:10'],
 6: ['18:30', '20:10'],
 7: ['20:15', '21:55'],
 8: ['17:10', '18:50']}

In [23]:
df_lec['Time'] = df_lec[0].str[1].map(d)
df_lec['Start Time'] = df_lec['Time'].str[0]+':00'
df_lec['End Time'] = df_lec['Time'].str[1]+':00'

df_lec['Subject'] = df_lec['title'].str.split('」').str[0].str.replace('\n','').str.replace('「','-')
df_lec['Description'] = '講師: ' + df_lec['title'].str.split('」').str[1]

df_lec = df_lec[['Subject',  'Start Date', 'End Date',  'Start Time', 'End Time', 'Description']]

df_lec = df_lec.drop_duplicates().reset_index(drop=True)

df_lec = df_lec.groupby(['Subject',  'Start Date', 'End Date','Description']).agg({'Start Time':min, 'End Time':max}).reset_index()

df_lec = df_lec[['Subject', 'Start Date', 'End Date', 'Start Time', 'End Time', 'Description']]

check diff

In [24]:
import difflib

In [25]:
import numpy as np

In [26]:
df_lec_prev = pd.read_csv('../jaist-lecture.csv', dtype=str).fillna('')

In [27]:
df_lec = df_lec.sort_values(['Subject', 'Start Date']).reset_index(drop=True)
df_lec_prev = df_lec_prev.sort_values(['Subject', 'Start Date']).reset_index(drop=True)

In [28]:
d = difflib.Differ()

diff = d.compare([' '.join(l) for l in df_lec_prev.values],
                 [' '.join(l) for l in df_lec.values])

In [29]:
print('\n'.join(diff))

- I211-数理論理学 2021/10/30 2021/10/30 09:20:00 12:50:00 講師: 小川
- I211-数理論理学 2021/11/06 2021/11/06 09:20:00 12:50:00 講師: 小川
- I211-数理論理学 2021/11/13 2021/11/13 09:20:00 12:50:00 講師: 小川
- I211-数理論理学 2021/11/20 2021/11/20 09:20:00 12:50:00 講師: 小川
+ I214-システム最適化 2023/01/14 2023/01/14 13:50:00 17:20:00 講師: （平石・金子）
+ I214-システム最適化 2023/01/21 2023/01/21 13:50:00 17:20:00 講師: （平石・金子）
+ I214-システム最適化 2023/01/28 2023/01/28 13:50:00 17:20:00 講師: （平石・金子）
+ I214-システム最適化 2023/02/11 2023/02/11 13:50:00 17:20:00 講師: （平石・金子）
+ I214-システム最適化 2023/02/18 2023/02/18 13:50:00 17:20:00 講師: （平石・金子）
+ I214-システム最適化 2023/03/04 2023/03/04 13:50:00 17:20:00 講師: （平石・金子）
+ I214-システム最適化 2023/03/11 2023/03/11 13:50:00 17:20:00 講師: （平石・金子）
+ I214-システム最適化 2023/03/18 2023/03/18 13:50:00 17:20:00 講師: （平石・金子）
+ I217-関数プログラミング 2022/06/12 2022/06/12 13:50:00 17:20:00 講師: （緒方）
+ I217-関数プログラミング 2022/06/19 2022/06/19 13:50:00 17:20:00 講師: （緒方）
+ I217-関数プログラミング 2022/07/10 2022/07/10 13:50:00 17:20:00 講師: （緒方）
+ I217-関数プログラミング 2022/07/1

In [30]:
df_lec = df_lec.sort_values(['Start Date', 'Start Time', "Subject"]).reset_index(drop=True)

In [31]:
df_lec.to_csv('../jaist-lecture.csv', index=False)